# Simple Fight 1v1 v101
This is a notebook what is used for the articles on my blog. The main intention to write the series of articles how to use python in game designer work. This will be updated.

# Simple Fight Game

The players summon different units, they move to each other and fight.

## 1:1 Fight
For the fight we want to use simulation to determine the outcome of the fight. Instead of using just formulas we are building simplified game-loop. Lets say units start on opposite sides of the field. Move to each other and attacks when its possible.
For that we need a way to save their state. For that we just use a dictionary for simplicity. This notebook covers 1:1 fight.

## Implementation

The current implementation and chosen data structures are not the best, but they are simple and easy to understand for the first steps. The main goal is to show how to use Python in-game designer work. The code is not optimized and is only for learning purposes.

In [5]:
# libraries and imports
import pandas as pd

# Some init constants and unit attributes
# For unit names we are using 'hyphen-case'
# For unit attributes we are using 'snake_case'
UNIT_TYPES = ['soldier', 'knight', 'archer', 'goblin', 'ork', 'axe-thrower']
# health - maximum health points
# attack_damage - damage per attack
# speed - how fast unit can move, m/s
# attack_range - how far unit can attack, m
# attack_cooldown - how often unit can attack, s
# Units attributes
UNITS_ATTRIBUTES = pd.DataFrame({
    'unit_type': UNIT_TYPES,
    'health': [100, 150, 50, 50, 100, 50],
    'attack_damage': [10, 15, 20, 10, 15, 20],
    'speed': [1.5, 1.2, 1, 1.5, 1.2, 1],
    'attack_range': [1, 1, 10, 1, 1, 10],
    'attack_cooldown': [0.7, 1, 2, 0.5, 1, 2]
}).set_index('unit_type')

class Unit:
    def __init__(self, unit_type: str, x=0):
        self.type = unit_type
        self.attributes = UNITS_ATTRIBUTES.loc[unit_type]
        self.max_health = self.attributes.health
        self.current_health = self.max_health
        self.current_attack_cooldown = 0
        self.x = x

In [9]:
DT = 0.01
FIELD_SIZE = 15

def create_unit(unit_type: str, x=0) -> Unit:
    return Unit(unit_type, x)

def distance_between(unit1: Unit, unit2: Unit):
    return abs(unit1.x - unit2.x)

def enemy_in_range(unit: Unit, enemy: Unit):
    return distance_between(unit, enemy) <= unit.attributes.attack_range

def unit_can_attack(unit: Unit):
    return unit.current_attack_cooldown <= 0

def simulate_fight(unit_type1: str, unit_type2: str):
    unit1 = create_unit(unit_type1, 0)
    unit2 = create_unit(unit_type2, FIELD_SIZE)
    time = 0
    while unit1.current_health > 0 and unit2.current_health > 0:
        if enemy_in_range(unit1, unit2):
            if unit_can_attack(unit1):
                unit2.current_health -= unit1.attributes.attack_damage
                unit1.current_attack_cooldown = unit1.attributes.attack_cooldown
        else:
            unit1.x += unit1.attributes.speed * DT
        if enemy_in_range(unit2, unit1):
            if unit_can_attack(unit2):
                unit1.current_health -= unit2.attributes.attack_damage
                unit2.current_attack_cooldown = unit2.attributes.attack_cooldown
        else:
            unit2.x -= unit2.attributes.speed * DT
        time += DT
        unit1.current_attack_cooldown = max(0, unit1.current_attack_cooldown - DT)
        unit2.current_attack_cooldown = max(0, unit2.current_attack_cooldown - DT)
    return time, unit1, unit2

def print_results(results):
    for time, unit1, unit2 in results:
        print('-' * 50)
        time = round(time, 1)
        print(f'Fight between {unit1.type} and {unit2.type} lasted {time} seconds')
        if unit1.current_health > 0:
            print(f'Winner: {unit1.type} with remaining health {unit1.current_health}')
        elif unit2.current_health > 0:
            print(f'Winner: {unit2.type} with remaining health {unit2.current_health}')
        else:
            print('Draw, both units died')

# Results are presented as a list of tuples (time, unit1, unit2)
# we want to print it as ASCII table where rows and columns are unit types
# cells are remaining health of unit1 after fight
def print_results_table(results):
    # first we need to find all unique unit types
    INTENT = 12
    unit_types = set()
    for _, unit1, unit2 in results:
        unit_types.add(unit1.type)
        unit_types.add(unit2.type)
    unit_types = sorted(list(unit_types))
    # print header
    print(' ' * INTENT, end='')
    for unit_type in unit_types:
        print(f'{unit_type:>{INTENT}}', end='')
    print()
    # print rows
    for unit_type1 in unit_types:
        print(f'{unit_type1:>{INTENT}}', end='')
        for unit_type2 in unit_types:
            # find result for this pair of unit types
            for time, u1, u2 in results:
                if u1.type == unit_type1 and u2.type == unit_type2:
                    break
            else:
                raise Exception(f'No result for {unit_type1} vs {unit_type2}')
            # print cell
            hp = max(u1.current_health, 0)
            if hp == 0:
                print(f'\x1b[31m{hp:>{INTENT}.0f}\x1b[0m', end='')
            else:
                print(f'\x1b[32m{hp:>{INTENT}.0f}\x1b[0m', end='')
        print()





In [8]:

results = []
for unit_type1 in UNIT_TYPES:
    for unit_type2 in UNIT_TYPES:
        t, u1, u2 = simulate_fight(unit_type1, unit_type2)
        results.append((t, u1, u2))

print_results_table(results)
print_results(results)

                  archer axe-thrower      goblin      knight         ork     soldier
      archer           0           0          50           0          35          20
 axe-thrower           0           0          50           0          35          20
      goblin           0           0           0           0           0           0
      knight          30          30          80           0          45          60
         ork           0           0          30           0           0          10
     soldier           0           0          40           0           0           0
--------------------------------------------------
Fight between soldier and soldier lasted 11.0 seconds
Draw, both units died
--------------------------------------------------
Fight between soldier and knight lasted 11.2 seconds
Winner: knight with remaining health 60.0
--------------------------------------------------
Fight between soldier and archer lasted 10.0 seconds
Winner: archer with remainin